In [25]:
import pandas as pd
import os
import json
import math
import datetime
import numpy as np
import xgboost as xgb

In [26]:
data_path = r'E:\data\data_set_phase1'
test_queries_data_path = os.path.join(data_path,'test_queries.csv')
test_plans_data_path = os.path.join(data_path,'test_plans.csv')
profiles_data_path = os.path.join(data_path,'profiles.csv')

train_queries_data_path = os.path.join(data_path,'train_queries.csv')
train_plans_data_path = os.path.join(data_path,'train_plans.csv')
train_clicks_data_path = os.path.join(data_path,'train_clicks.csv')

## df表的连接和简单缺失值填补

In [27]:
df_test_queries = pd.read_csv(test_queries_data_path)
df_test_plans = pd.read_csv(test_plans_data_path)
df_profiles = pd.read_csv(profiles_data_path)

df_train_queries = pd.read_csv(train_queries_data_path)
df_train_plans = pd.read_csv(train_plans_data_path)
df_train_clicks = pd.read_csv(train_clicks_data_path)

In [28]:
df_train = pd.merge(df_train_queries,df_train_clicks,how='left',on=['sid'])
df_train = pd.merge(df_train, df_train_plans, how='left', on=['sid'])
df_train = pd.merge(df_train, df_profiles, how='left', on=['pid'])
df_train['click_mode'] = df_train.click_mode.fillna(0)
# df_train = df_train[df_train['click_mode'] !=0 ]
df_train['plans'] = df_train.plans.fillna("0")
# df_train = df_train.dropna(axis=0)

df_test = pd.merge(df_test_queries,df_test_plans,how='left',on=['sid'])
df_test = pd.merge(df_test,df_profiles,how='left',on=['pid'])
df_test['plans'] = df_test.plans.fillna("0")
# df_test = df_test.drop(['pid','req_time','o','d','plan_time'],axis=1)
# df_test = df_test.fillna(method='ffill')
# df_test = df_test.fillna(method='bfill')

## 处理od数据，
 - o,d -> o1(经度),o2（纬度）,d1,d2
 - 将o d 的经纬度值放大，直接计算和城中心的距离

In [30]:
def s2tuple(s):
    s = s.split(",")
    s = tuple(map(float,s))
    return s
def dis(x1,y1,x2,y2):
    return math.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2))
    
def clean_od(df):
#     print(df.count())
    df['o'] = df.o.apply(s2tuple)
    df['d'] = df.d.apply(s2tuple)
    olis = list(df['o'])
    dlis = list(df['d'])
    olis = list(map(lambda x:(x[0]*100,x[1]*100),olis))
    dlis = list(map(lambda x:(x[0]*100,x[1]*100),dlis))
    num = len(olis)
#     print(olis)
    center_1 = 3391
    center_0 = 11640
    df['o1'] = [i[0] for i in olis]
    df['o2'] = [i[1] for i in olis]
    df['d1'] = [i[0] for i in dlis]
    df['d2'] = [i[1] for i in dlis]
    df['o2center'] = [dis(olis[i][0],olis[i][1],center_0,center_1) for i in range(num)]
    df['d2center'] = [dis(dlis[i][0],dlis[i][1],center_0,center_1) for i in range(num)]
    
clean_od(df_train)
clean_od(df_test)
df_train = df_train.drop(['o','d'],axis=1)
df_test = df_test.drop(['o','d'],axis=1)

## 处理时间req_time,删除剩余的time列
 - month, day, hour，minute, second, weekday,isweekend

In [31]:
def clean_req_time(s):
    a = datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    res = [a.date().__getattribute__('month'),a.date().__getattribute__('day'),a.time().__getattribute__('hour'),a.time().__getattribute__('minute'),a.time().__getattribute__('second'),a.weekday(),int(a.weekday() > 4)]
    return res
df_train['req_time'] = df_train.req_time.apply(clean_req_time)
df_test['req_time'] = df_test.req_time.apply(clean_req_time)
for i in range(7):
    s = 'req_time'
    df_train['req_time'+str(i)] = [li[i] for li in df_train['req_time']]
    df_test['req_time'+str(i)] = [li[i] for li in df_test['req_time']]
df_train = df_train.drop(['req_time','click_time','plan_time'],axis=1)
df_test = df_test.drop(['req_time','plan_time'],axis=1)

In [32]:
df_train.head()

,sid,pid,click_mode,plans,p0,p1,p2,p3,p4,p5,...,d2,o2center,d2center,req_time0,req_time1,req_time2,req_time3,req_time4,req_time5,req_time6
0,3000821,NaN,9.0,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,NaN,NaN,NaN,NaN,...,3996.0,606.099827,605.052890,11,2,17,54,30,4,0
1,3085857,210736.0,1.0,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0.0,0.0,0.0,0.0,1.0,0.0,...,3979.0,593.000843,588.041665,11,16,10,53,10,4,0
2,2944522,NaN,9.0,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,NaN,NaN,NaN,NaN,...,4000.0,602.067272,609.138736,10,6,10,33,58,5,1
3,559931,202427.0,1.0,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1.0,0.0,0.0,0.0,0.0,0.0,...,3990.0,597.141524,599.000835,11,23,14,54,11,4,0
4,2819352,172251.0,7.0,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0.0,0.0,0.0,1.0,0.0,0.0,...,3986.0,605.029751,595.007563,10,30,11,48,41,1,0


## 处理plan
 - 1.用训练和测试数据集中已存在的mode，distance，eta，price，拟合缺失的price。(不好，决策树对缺失值不敏感)
 - 2.每个mode转为3列，distance，eta和mode，没有的mode这三列填充为inf = 10000000（一个极大值）
 - 3.将plan转化为一个list，包含可选的mode

In [33]:
train_plan_ls = list(df_train.plans)
train_plan_ls = [json.loads(p) for p in train_plan_ls]

test_plan_ls = list(df_test.plans)
test_plan_ls = [json.loads(p) for p in test_plan_ls]

all_plan_ls = train_plan_ls + test_plan_ls

# gbdt_x = []
# gbdt_y = []
# from sklearn.ensemble import GradientBoostingRegressor
# gbdt = GradientBoostingRegressor()
# for p in all_plan_ls:
#     if p == 0:
#         continue
#     for pdc in p:
#         if pdc['price']!="":
#             dis = int(pdc['distance'])
#             eta = int(pdc['eta'])
#             mode = int(pdc['transport_mode'])
#             price = int(pdc['price'])
#             gbdt_x.append([dis,eta,mode])
#             gbdt_y.append(price)
# print('sta')
# gbdt.fit(gbdt_x,gbdt_y)
# print('fi')

In [34]:
def predict_price(plan_ls):
    for p in plan_ls:
        if p == 0:
            continue
        for pdc in p:
            if pdc['price']=="":
                dis = int(pdc['distance'])
                eta = int(pdc['eta'])
                mode = int(pdc['transport_mode'])
                pdc['price'] = gbdt.predict([[dis,eta,mode]])[0]

In [35]:
def plans2df_cols(df,plan_ls):
    inf = float('NaN')
    mode_dic = {}
    for i in range(0,66):
        mode_dic[i]=[]
    # for i in range(0,33):
    #     mode_dic[i].append(inf)
    # mode_dic
    for p in plan_ls:
        for i in range(0,66):
            mode_dic[i].append(inf)
        if p == 0:
            continue
        for pdc in p:
                dis = float(pdc['distance'])
                eta = float(pdc['eta'])
                mode = int(pdc['transport_mode'])
                mode_dic[(mode-1)*6+0][-1] = dis
                mode_dic[(mode-1)*6+1][-1] = eta
                mode_dic[(mode-1)*6+3][-1] = dis/eta
                if pdc['price']!="":
                    price = int(pdc['price'])
                    mode_dic[(mode-1)*6+2][-1] = price
                    mode_dic[(mode-1)*6+4][-1] = dis/price
                    mode_dic[(mode-1)*6+5][-1] = eta/price
    for i in range(0,66):
        df['plan'+str(i)] = mode_dic[i]
        mode_dic[i] = []

In [36]:
def plan2mode(s):
    plan_lis = json.loads(s)
    if plan_lis == 0:
        return list(range(12))
    pl = []
    for p in plan_lis:
        pl.append(int(p['transport_mode']))
    return pl

In [37]:
# predict_price(train_plan_ls)
# predict_price(test_plan_ls)

plans2df_cols(df_train,train_plan_ls)
plans2df_cols(df_test,test_plan_ls)

df_train['plans'] = df_train.plans.apply(plan2mode)
df_test['plans'] = df_test.plans.apply(plan2mode)

In [38]:
df_train['sid'] = df_train.sid.apply(int)
df_train['click_mode'] = df_train.click_mode.apply(int)

df_test['sid'] = df_test.sid.apply(int)

df_train = df_train.drop(['pid'],axis=1)
df_test = df_test.drop(['pid'],axis=1)

In [39]:
for i in df_train.columns:
    print(i)
for i in df_test.columns:
    print(i)

sid
click_mode
plans
p0
p1
p2
p3
p4
p5
p6
p7
p8
p9
p10
p11
p12
p13
p14
p15
p16
p17
p18
p19
p20
p21
p22
p23
p24
p25
p26
p27
p28
p29
p30
p31
p32
p33
p34
p35
p36
p37
p38
p39
p40
p41
p42
p43
p44
p45
p46
p47
p48
p49
p50
p51
p52
p53
p54
p55
p56
p57
p58
p59
p60
p61
p62
p63
p64
p65
o1
o2
d1
d2
o2center
d2center
req_time0
req_time1
req_time2
req_time3
req_time4
req_time5
req_time6
plan0
plan1
plan2
plan3
plan4
plan5
plan6
plan7
plan8
plan9
plan10
plan11
plan12
plan13
plan14
plan15
plan16
plan17
plan18
plan19
plan20
plan21
plan22
plan23
plan24
plan25
plan26
plan27
plan28
plan29
plan30
plan31
plan32
plan33
plan34
plan35
plan36
plan37
plan38
plan39
plan40
plan41
plan42
plan43
plan44
plan45
plan46
plan47
plan48
plan49
plan50
plan51
plan52
plan53
plan54
plan55
plan56
plan57
plan58
plan59
plan60
plan61
plan62
plan63
plan64
plan65
sid
plans
p0
p1
p2
p3
p4
p5
p6
p7
p8
p9
p10
p11
p12
p13
p14
p15
p16
p17
p18
p19
p20
p21
p22
p23
p24
p25
p26
p27
p28
p29
p30
p31
p32
p33
p34
p35
p36
p37
p38
p39
p40
p41
p42
p

In [40]:
df_train.to_csv(os.path.join(data_path,'f145_with0_all_train.csv'),index=False)
# df_train[400000:].to_csv(os.path.join(data_path,'f145_valid.csv'),index=False)
# df_test.to_csv(os.path.join(data_path,'f145_test.csv'),index=False)

## xgboost 训练和测试

In [76]:
# click_mode = list(df_train.click_mode)
# param = {}
# param['num_class'] = 12
# param['objective'] = 'multi:softprob'
# # param = {}
# param['eta'] = 0.3
# param['max_depth'] = 9
# param['gamma'] = 0.03
# param['colsample_bytree'] = 0.8
# param['subsample'] = 0.85
# # param['silent'] = 1
# # param['nthread'] = 4
# # param['num_class'] = 3
# # num_round = 50
# # param['objective'] = 'multi:softprob'
# num_round = 2
# bst = xgb.train(param,xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1),label=click_mode),num_round)
# pred = bst.predict(xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1)))
# pred = pred.tolist()

# train_plan_mode_lis = list(df_train['plans'])
# print(train_plan_mode_lis[:5])
# pred_label = []
# # sorted(list(zip(range(0,12),pred[0])),key = lambda x:x[1],reverse=True)
# for i in range(len(pred)):
#     sort_pred = sorted(list(zip(range(0,12),pred[i])),key = lambda x:x[1],reverse=True)
#     pred_label.append(0)
#     for prob in sort_pred:
#         if prob[0] in train_plan_mode_lis[i]:
#             pred_label[-1] = prob[0]
#             break
# pro = [pred_label[i]==click_mode[i] for i in range(len(pred_label))]
# pro = list(map(int,pro))
# print(np.sum(pro),len(pred_label),np.sum(pro)/len(pred_label))

[[9, 3, 4, 6, 2, 5], [7, 3, 4, 9, 1], [9, 3, 4, 10, 2, 6, 7], [10, 3, 4, 7, 1], [7, 3, 4, 7]]
340868 500000 0.681736


In [81]:
# xgb.plot_importance(bst)
# import matplotlib.pyplot as plt
# s = bst.get_fscore()
# for i in s:
#     if s[i]<10:
#         print(i,s[i])

In [19]:
cv = 450000
train_click_mode = list(df_train.click_mode)[:cv]
valid_click_mode = list(df_train.click_mode)[cv:]
valid_plan_mode_lis = list(df_train['plans'])[cv:]

for k in range(3,15):    
    param = {}
    param['num_class'] = 12
    param['objective'] = 'multi:softprob'
    # param = {}
    param['eta'] = 0.3
    param['max_depth'] = k
    param['gamma'] = 0
    param['colsample_bytree'] = 1
    param['subsample'] = 1
    # param['silent'] = 1
    # param['nthread'] = 4
    # param['num_class'] = 3
    # num_round = 50
    # param['objective'] = 'multi:softprob'
    num_round = 10
    bst = xgb.train(param,xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1).values[:cv],label=train_click_mode),num_round)
    pred = bst.predict(xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1).values[cv:]))
    pred = pred.tolist()

    # valid_plan_mode_lis = list(df_train['plans'])[cv:]
    # print(train_plan_mode_lis[:5])

    pred_label = []
    # sorted(list(zip(range(0,12),pred[0])),key = lambda x:x[1],reverse=True)
    for i in range(len(pred)):
        sort_pred = sorted(list(zip(range(0,12),pred[i])),key = lambda x:x[1],reverse=True)
        for prob in sort_pred:
            if prob[0] in valid_plan_mode_lis[i] or prob[0] == 0:
                pred_label.append(prob[0])
                break
    pro = [pred_label[i]==valid_click_mode[i] for i in range(len(pred_label))]
    pro = list(map(int,pro))
    print(k,np.sum(pro),len(pred_label),np.sum(pro)/len(pred_label))

3 42687 51413 0.8302763892400755
4 43069 51413 0.8377064166650459
5 43504 51413 0.8461673117693969


KeyboardInterrupt: 

In [20]:
pred[:10]

[[0.010404307395219803,
  0.2272443175315857,
  0.010405348613858223,
  0.010404307395219803,
  0.010404307395219803,
  0.010400209575891495,
  0.010404307395219803,
  0.6686846017837524,
  0.010404307395219803,
  0.010435434989631176,
  0.010404307395219803,
  0.010404307395219803],
 [0.010913572274148464,
  0.010933917947113514,
  0.010914665646851063,
  0.010913572274148464,
  0.010913572274148464,
  0.01090926956385374,
  0.010913572274148464,
  0.22381147742271423,
  0.010913572274148464,
  0.667035698890686,
  0.010913572274148464,
  0.010913572274148464],
 [0.013557310216128826,
  0.013582584448158741,
  0.5144663453102112,
  0.013557310216128826,
  0.013557310216128826,
  0.013551969081163406,
  0.013557310216128826,
  0.042329948395490646,
  0.013557310216128826,
  0.3211679756641388,
  0.013557310216128826,
  0.013557310216128826],
 [0.009847849607467651,
  0.1645725816488266,
  0.009848840534687042,
  0.009847849607467651,
  0.009847849607467651,
  0.009843971580266953,
  0.

In [ ]:
# num_round:
# 50 ->0.65618
# eta:
# 0.1 -> 0.65124
# 0.2 ->0.65618
# 0.3
# max_depth
# 
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# import matplotlib.pyplot as plt
# click_mode = list(df_train['click_mode'])
# param_grid = {
#     #'subsample': np.linspace(0.3, 1, 5),
#     #'colsample_bytree': np.arange(0.3,1,5)
#     #'min_child_weight':np.arange(1,16)
#     #'reg_alpha':[i*0.0 for i in range(10)]
#     #'gamma':np.linspace(0,0.2,30)
#     #'learning_rate':np.linspace(0.05,0.2,10)
# #     'n_estimators': [i*100 for i in np.arange(1,4)]
# #     'subsample': np.linspace(0.8,1,20)
#     #'colsample_bytree': [i*0.05 for i in np.arange(16,21)]
# }

# df_1 = df_train.drop(['sid','click_mode','plans'],axis=1)
# # for i in range(10,31,10):
# #     xgbc = XGBClassifier(n_estimators=i)
# #     grid = GridSearchCV(xgbc, param_grid, scoring='accuracy')
# #     print(df_train.drop(['sid','plans','click_mode'],axis=1).values.shape,len(click_mode))
# #     grid.fit(df_train.drop(['sid','plans','click_mode'],axis=1).values,click_mode)
# # #     x.append(i)
# # #     y.append(grid.best_score_)
# #     print(i,grid.best_score_)

## 测试集预测&&结果保存csv

In [ ]:
res = bst.predict(xgb.DMatrix(df_test.drop(['sid','plans'],axis=1).values))
res = res.tolist()
test_plan_mode_lis = list(df_test['plans'])
res_label = []
# sorted(list(zip(range(0,12),pred[0])),key = lambda x:x[1],reverse=True)
for i in range(len(res)):
    sort_pred = sorted(list(zip(range(0,12),res[i])),key = lambda x:x[1],reverse=True)
    for prob in sort_pred:
        if prob[0] in test_plan_mode_lis[i] or prob[0] == 0:
            res_label.append(prob[0])
            break
res_label[:5]

In [ ]:
res_df = pd.DataFrame({'sid':df_test['sid'],'recommend_mode':res_label})

In [ ]:
res_df.to_csv(os.path.join(data_path,'xbg_no_fillna_f145.csv'),quoting=1,index=0)

In [41]:
his_df2 = pd.read_csv(r'E:\kdd-cup-2019\Context-Aware-Multi-Modal-Transportation-Recommendation-master\submit\lgb_result_2019-04-23-09-57-48.csv')
# his_df1 = pd.read_csv(os.path.join(data_path,'xbg_no_fillna_f145_0.2_50.csv'))
his_ans1 = list(his_df1['recommend_mode'])
his_df2 = pd.read_csv(os.path.join(data_path,r'sk_xgb_10_10_0.1.csv'))
his_ans2 = list(his_df2['recommend_mode'])
same = [int(his_ans2[i] == his_ans1[i]) for i in range(len(his_ans1))]
np.sum(same)/len(his_ans1)
# his_df2 = pd.read_csv(r'E:\kdd-cup-2019\Context-Aware-Multi-Modal-Transportation-Recommendation-master\submit\lgb_result_2019-04-23-09-57-48.csv')

0.9191059581593506

In [42]:
for i in range(0,12):
    print(i,his_ans1.count(i))

0 1787
1 17042
2 33117
3 400
4 38
5 10897
6 224
7 21126
8 158
9 7300
10 1553
11 716


In [43]:
for i in range(0,12):
    print(i,his_ans2.count(i))

0 0
1 16885
2 33598
3 89
4 1789
5 11027
6 62
7 23229
8 34
9 6683
10 547
11 415


In [6]:
# his_df2.to_csv(os.path.join(data_path,'baseline_lgb.csv'),quoting=1,index=0)